# PROJET IA for HumanForYou

|Auteur|
|---|
|G. DUBOYS DE LAVIGERIE|
|T. VILETTE|
|O. BOUSSARD|
|A. BRICON|

## Objectifs du Livrable

Le présent document constitue le premier livrable du projet IA pour HumanForYou, visant à analyser et traiter les différentes données fournies par l'entreprise dans le but de comprendre les facteurs influençant le taux de rotation des employés. Les objectifs principaux de ce livrable sont les suivants :

1. **Compréhension des données :** Explorer et comprendre les différentes sources de données fournies par HumanForYou, y compris les données des ressources humaines, les évaluations des managers, les enquêtes sur la qualité de vie au travail et les horaires de travail.

2. **Prétraitement des données :** Nettoyer les données en éliminant les valeurs manquantes, en identifiant et en traitant les éventuelles erreurs ou incohérences, et en préparant les données pour l'analyse et la modélisation.

3. **Exploration des données :** Effectuer une analyse exploratoire des données pour identifier les tendances, les relations et les motifs significatifs pouvant influencer le taux de rotation des employés.

4. **Visualisation des données :** Utiliser des techniques de visualisation de données pour représenter graphiquement les principales caractéristiques des données et faciliter la compréhension des résultats.

5. **Préparation des données pour la modélisation :** Préparer les données en sélectionnant les variables pertinentes, en transformant les variables catégorielles en variables numériques, et en divisant les données en ensembles d'entraînement et de test pour la modélisation.

## Attendus

À la fin de ce livrable, nous nous attendons à ce que les données fournies par HumanForYou soient prêtes à être analysées et utilisées pour la modélisation. Nous fournirons une analyse détaillée de la qualité des données, des tendances et des relations identifiées, ainsi qu'une documentation complète sur les étapes de prétraitement des données.

### 1. Préparation de l'environnement

Pour faciliter l'importation des bibliothèques nécessaires au bon fonctionnement du code, veuillez exécuter le fichier `setup.ipynb`. Ce fichier s'occupe de configurer l'environnement en important toutes les librairies essentielles. 

In [112]:
# compatibilité python 2 et python 3
from __future__ import division, print_function, unicode_literals

# imports
import numpy as np
import os

# stabilité du notebook d'une exécution à l'autre
np.random.seed(42)

# jolies figures directement dans le notebook
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# où sauver les figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "workflowDS"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID) # le dossier doit exister

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# ignorer les warnings inutiles (voir SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

### 2. Import des données

Dans cette section, nous automatisons le processus d'importation des données en implémentant une fonction qui effectue les étapes suivantes :
1. Téléchargement de l'archive contenant les fichiers.
2. Extraction des fichiers de l'archive.

Le code ci-dessous réalise le chargement des fichiers suivants :
- `employee_survey_data.csv`
- `general_data.csv`
- `in_time.csv`
- `out_time.csv`
- `manager_survey_data.csv`

De même, on va créer une fonction utilisant [`pandas`](https://pandas.pydata.org/) qui charge les données en mémoire dans un [`Pandas DataFrame`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html#pandas.DataFrame).

In [113]:
import os
import zipfile
from six.moves import urllib
import pandas as pd
from sklearn.impute import SimpleImputer

pd.set_option('display.max_columns', None)

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/"
REPO_PATH = "AnatholyB1/AI_A4/main/"
DATA_PATH = os.path.join("datasets", "all")
DATA_URL = DOWNLOAD_ROOT + REPO_PATH + "data.zip"


def fetch_data(data_url=DATA_URL, data_path=DATA_PATH):
    if not os.path.isdir(data_path):
        os.makedirs(data_path)
    zip_path = os.path.join(data_path, "data.zip")
    urllib.request.urlretrieve(data_url, zip_path)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_path)

fetch_data()

def load_employee_data(housing_path=DATA_PATH):
    csv_path = os.path.join(housing_path + "\employe", "employee_survey_data.csv")
    return pd.read_csv(csv_path)

def load_general_data(housing_path=DATA_PATH):
    csv_path = os.path.join(housing_path + "\general", "general_data.csv")
    return pd.read_csv(csv_path)

def load_in_time_data(housing_path=DATA_PATH):
    csv_path = os.path.join(housing_path + "\in_out_time", "in_time.csv")
    return pd.read_csv(csv_path)

def load_out_time_data(housing_path=DATA_PATH):
    csv_path = os.path.join(housing_path + "\in_out_time", "out_time.csv")
    return pd.read_csv(csv_path)

def load_manager_data(housing_path=DATA_PATH):
    csv_path = os.path.join(housing_path + "\manager", "manager_survey_data.csv")
    return pd.read_csv(csv_path)

employee = load_employee_data()
general = load_general_data()
in_time = load_in_time_data()
out_time = load_out_time_data()
manager = load_manager_data()

### 3. Analyse des différentes données

#### 3.1. employee

In [114]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Créez un pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
])


# Appliquez le pipeline à l'ensemble du DataFrame
employee_transformed = numeric_transformer.fit_transform(employee)

# Transformez le résultat du pipeline en DataFrame
employee_df = pd.DataFrame(employee_transformed, columns = employee.columns)
employee_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4410 non-null   float64
 1   EnvironmentSatisfaction  4410 non-null   float64
 2   JobSatisfaction          4410 non-null   float64
 3   WorkLifeBalance          4410 non-null   float64
dtypes: float64(4)
memory usage: 137.9 KB


#### 3.2 General

| **Variable**           | **Type de données** | **Nombre de valeurs Manquantes** | **Nature de la variable** | **Nombre de catégories uniques** |
|------------------------|---------------------|----------------------------------|---------------------------|----------------------------------|
| **Age**                | Quantitative        | 0                                | Ordinal                   | 35                               |
| **Attrition**          | Qualitative         | 0                                | Nominal (Booléen)         | 2                                |
| **BusinessTravel**     | Qualitative         | 0                                | Ordinal                   | 3                                |
| **Department**         | Qualitative         | 0                                | Nominal                   | 3                                |
| **DistanceFromHome**   | Quantitative        | 0                                | Ordinal                   | 16                               |
| **Education**          | Quantitative        | 0                                | Ordinal                   | 5                                |
| **EducationField**     | Qualitative         | 0                                | Nominal                   | 6                                |
| **EmployeeCount**      | Quantitative        | 0                                | Ordinal                   | 1                                |
| **EmployeeID**         | Quantitative        | 0                                | Ordinal                   | 4410                             |
| **Gender**             | Qualitative         | 0                                | Nominal (Booléen)         | 2                                |
| **JobLevel**           | Quantitative        | 0                                | Ordinal                   | 5                                |
| **JobRole**            | Qualitative         | 0                                | Nominal                   | 9                                |
| **MaritalStatus**      | Qualitative         | 0                                | Nominal                   | 3                                |
| **MonthlyIncome**      | Quantitative        | 0                                | Ordinal                   | 1349                             |
| **NumCompaniesWorked** | Quantitative        | 19                               | Ordinal                   | 10                               |
| **Over18**             | Qualitative         | 0                                | Nominal                   | 1                                |
| **PercentSalaryHike**  | Quantitative        | 0                                | Ordinal                   | 14                               |
| **StandardHours**      | Quantitative        | 0                                | Ordinal                   | 1                                |
| **StockOptionLevel**   | Quantitative        | 0                                | Ordinal                   | 4                                |
| **TotalWorkingYears**  | Quantitative        | 9                                | Ordinal                   | 29                               |
| **TrainingTimesLastYear** | Quantitative      | 0                                | Ordinal                   | 7                                |
| **YearsAtCompany**     | Quantitative        | 0                                | Ordinal                   | 29                               |
| **YearsSinceLastPromotion** | Quantitative    | 0                                | Ordinal                   | 15                               |
| **YearsWithCurrManager** | Quantitative      | 0                                | Ordinal                   | 16                               |

In [115]:
# Supprimer useless colonnes
general_dr = general.drop(['Over18', 'EmployeeCount', 'StandardHours'], axis=1)

# Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

general_num = general_dr.select_dtypes(include=[np.number]) 
num_attribs = list(general_num)

general_cat_ordinal = ['BusinessTravel']
general_cat_nominal = ['Attrition','Department','EducationField','Gender','JobRole','MaritalStatus']

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
])

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat_nom", OneHotEncoder(), general_cat_nominal),
    ("cat_ord", OrdinalEncoder(), general_cat_ordinal),
])

# Appliquer le pipeline au DataFrame initial
general_tr = full_pipeline.fit_transform(general_dr)

# Créer un DataFrame à partir du résultat
general_prepared = pd.DataFrame(general_tr, columns=full_pipeline.get_feature_names_out())


#### 3.3 In_time

In [116]:

def convert_all_to_datetime(df):
    year_dict = {}
    month_dict = {}
    day_dict = {}

    for column_name in df.columns:
        try:
            df[column_name] = pd.to_datetime(df[column_name])
            year_dict[column_name + '-year'] = df[column_name].dt.year
            month_dict[column_name + '-month'] = df[column_name].dt.month
            day_dict[column_name + '-day'] = df[column_name].dt.day
        except ValueError:
            # Skip columns that cannot be converted to datetime
            pass

    year_df = pd.DataFrame(year_dict)
    month_df = pd.DataFrame(month_dict)
    day_df = pd.DataFrame(day_dict)

    df = pd.concat([df, year_df, month_df, day_df], axis=1)
    return df

in_time = convert_all_to_datetime(in_time)
out_time = convert_all_to_datetime(out_time)

suprimmer les colones qui ne sont pas des dates

In [130]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

def convert_all_to_datetime(X,  Y):    
    cols_to_drop = X.filter(regex='-day$').columns
    X = X.drop(cols_to_drop, axis=1)
    cols_to_drop = X.filter(regex='-month$').columns
    X = X.drop(cols_to_drop, axis=1)
    cols_to_drop = X.filter(regex='-year$').columns
    X = X.drop(cols_to_drop, axis=1)


    first_column = X.columns[0]
    X = X.drop([first_column], axis=1)
    # Supprimer les colonnes où toutes les valeurs sont NaT
    X = X.dropna(axis=1, how='all')

    cols_to_drop = Y.filter(regex='-day$').columns
    Y = Y.drop(cols_to_drop, axis=1)
    cols_to_drop = Y.filter(regex='-month$').columns
    Y = Y.drop(cols_to_drop, axis=1)
    cols_to_drop = Y.filter(regex='-year$').columns
    Y = Y.drop(cols_to_drop, axis=1)


    first_column = Y.columns[0]
    Y = Y.drop([first_column], axis=1)

    # Supprimer les colonnes où toutes les valeurs sont NaT
    Y = Y.dropna(axis=1, how='all')


    # Calculer la médiane du temps horaire pour chaque personne
    median_time = (Y - X).median()

    for column in X.columns:
        # Trouver où les valeurs sont manquantes
        in_time_nan = X[column].isna()
        out_time_nan = Y[column].isna()

        # Si 'in_time' est manquant, soustraire la médiane du temps horaire à 'out_time'
        X.loc[in_time_nan, column] = X.loc[in_time_nan, column] - median_time[column]

        # Si 'out_time' est manquant, ajouter la médiane du temps horaire à 'in_time'
        Y.loc[out_time_nan, column] = Y.loc[out_time_nan, column] + median_time[column]
    
    return  Y - X



class PreTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.imputer = SimpleImputer(strategy="median")

    def fit(self, X, y=None):
        self.imputer.fit(X)
        return self

    def transform(self, X, y=None):
        for col in X.columns:
            X[col] = X[col].apply(lambda x: x.total_seconds() / 3600 if pd.notnull(x) else np.nan)    
        stats = pd.DataFrame()
        stats['mean'] = pd.Series(X.mean(axis=1))
        stats['median'] = pd.Series(X.median(axis=1))
        stats['min'] = pd.Series(X.min(axis=1))
        stats['max'] = pd.Series(X.max(axis=1))
        return stats, stats.columns
        # Convertir les timedelta en une quantité numérique représentant le nombre d'heures


# Définir le pipeline
num_pipeline = Pipeline([
    ('pre_transformer', PreTransformer()),
])

# Appliquer le pipeline
hourly_time_prepared, stats_col = num_pipeline.fit_transform(convert_all_to_datetime(in_time, out_time))

hourly_time_prepared = pd.DataFrame(hourly_time_prepared, columns= stats_col )

#Ajouter la colonne 'EmployeeID' pour pouvoir joindre les DataFrames
hourly_time_prepared['EmployeeID'] = employee_df['EmployeeID']


,mean,median,min,max,EmployeeID
0,7.373651,7.397222,6.504722,7.948333,1.0
1,7.718969,7.718750,6.725278,8.558889,2.0
2,7.013240,6.982639,6.198333,7.993611,3.0
3,7.193678,7.179722,6.280556,7.890833,4.0
4,8.006175,7.963611,7.310278,8.828333,5.0


#### Jointure

In [118]:
# Utiliser la fonction merge pour faire la première jointure avec general
result = pd.merge(employee_df, manager, on='EmployeeID')

# Utiliser la fonction merge pour faire la deuxième jointure avec general_prepared
result = pd.merge(result, general_prepared, left_on='EmployeeID', right_on='num__EmployeeID')

result.head()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,num__Age,num__DistanceFromHome,num__Education,num__EmployeeID,num__JobLevel,num__MonthlyIncome,num__NumCompaniesWorked,num__PercentSalaryHike,num__StockOptionLevel,num__TotalWorkingYears,num__TrainingTimesLastYear,num__YearsAtCompany,num__YearsSinceLastPromotion,num__YearsWithCurrManager,cat_nom__Attrition_No,cat_nom__Attrition_Yes,cat_nom__Department_Human Resources,cat_nom__Department_Research & Development,cat_nom__Department_Sales,cat_nom__EducationField_Human Resources,cat_nom__EducationField_Life Sciences,cat_nom__EducationField_Marketing,cat_nom__EducationField_Medical,cat_nom__EducationField_Other,cat_nom__EducationField_Technical Degree,cat_nom__Gender_Female,cat_nom__Gender_Male,cat_nom__JobRole_Healthcare Representative,cat_nom__JobRole_Human Resources,cat_nom__JobRole_Laboratory Technician,cat_nom__JobRole_Manager,cat_nom__JobRole_Manufacturing Director,cat_nom__JobRole_Research Director,cat_nom__JobRole_Research Scientist,cat_nom__JobRole_Sales Executive,cat_nom__JobRole_Sales Representative,cat_nom__MaritalStatus_Divorced,cat_nom__MaritalStatus_Married,cat_nom__MaritalStatus_Single,cat_ord__BusinessTravel
0,1.0,3.0,4.0,2.0,3,3,51.0,6.0,2.0,1.0,1.0,131160.0,1.0,11.0,0.0,1.0,6.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0
1,2.0,3.0,2.0,4.0,2,4,31.0,10.0,1.0,2.0,1.0,41890.0,0.0,23.0,1.0,6.0,3.0,5.0,1.0,4.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2,3.0,2.0,2.0,1.0,3,3,32.0,17.0,4.0,3.0,4.0,193280.0,1.0,15.0,3.0,5.0,2.0,5.0,0.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,4.0,4.0,4.0,3.0,2,3,38.0,2.0,5.0,4.0,3.0,83210.0,3.0,11.0,3.0,13.0,5.0,8.0,7.0,5.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5.0,4.0,1.0,3.0,3,3,32.0,10.0,1.0,5.0,1.0,23420.0,4.0,12.0,2.0,9.0,2.0,6.0,0.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0
